In [4]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd

from glob import glob


In [5]:
data = []
for path in glob("./Annotations/*.xml"):
    parse = ET.parse(path)
    objects = [[i.tag,i.text] for i in parse.iter()]
    image = {i:j for i ,j in objects[:11]}
    boxes = []
    for ob,name,pose,_,__,___,xmin,ymin,xmax,ymax in np.array(objects[11:]).reshape(-1,20)[:,1::2]:
        boxes.append(dict(
            name=name,
            xmin=int(xmin),
            ymin=int(ymin),
            xmax=int(xmax),
            ymax=int(ymax),
        ))    
    image['boxes'] = boxes
    image['filename'] = f"./JPEGImages/{image['filename']}"
    [image.pop(i) for i in ['annotation','path','folder','source','size','database','segmented','depth']];
    data.append(image)
    
    
rows = []

for row in data.copy():
    for box in row['boxes']:
        box['filename'] = row['filename']
        box['width'] = row['width']
        box['height'] = row['height']
        box['class'] = box.pop('name')
        rows.append(box)
        
train = pd.DataFrame(rows)[['filename','width','height','class','xmin','ymin','xmax','ymax']]
train.to_csv("./data.csv",index=False)